# Patient Cohort Analysis

This analysis was done on a cohort of blood cancer patients for submission to a medical conference. The research team was trying to assess sequential treatment and responses on patients. All sensitive data omitted. 

In [1]:
import pandas as pd
import numpy as np

LOTdata dataframe contains information on lines of therapy, order, start/end dates, response and NCT for each patient.

In [2]:
# LOTdata from All4Cure - Manually Curated

LOTdata = pd.read_csv("data.csv")
LOTdata = LOTdata.rename(columns = {'Patient':'A4CUID'})
LOTdata = LOTdata[['A4CUID', 'Order', 'Treatment Start', 'Line of Therapy', 'End Treatment', 'Date Best Response', 'Response', 'NCT number']]

ptkey dataframe contains information on diagnosis, location, date of diagnosis, age, gender, status (alive or deceased) and clinic information. 

In [3]:
# patient key - from ptkey spreadsheet 
ptkey = pd.read_csv("data2.csv")

In [4]:
#get a count of A4CUID by diagnosis 
ptkey.groupby(['diagnosis'])['A4CUID'].count()

diagnosis
Amyloidosis                   2
Lymphoma                      2
MGUS                         25
MM                          409
Myelodysplastic Syndrome      2
PCL                           8
SMM                          51
Waldenstrom's                 2
Name: A4CUID, dtype: int64

### Basic Demographics

In [5]:
# Exclude Lymphoma, Amyloidosis, Myelodysplastic Syndrome and Waldenstroms from analysis
MM_PCL_SMM_MGUS = ptkey.loc[~ptkey['diagnosis'].str.contains('Lymphoma|Amyloidosis|Myelodys|Walden')]
print(MM_PCL_SMM_MGUS.groupby(['diagnosis'])['A4CUID'].count())

MM_PCL_SMM_MGUS_SUM = MM_PCL_SMM_MGUS.groupby(['diagnosis'])['A4CUID'].count().sum()
print("Total patients (MGUS, SMM, MM, PCL) = {}".format(MM_PCL_SMM_MGUS_SUM))

diagnosis
MGUS     25
MM      409
PCL       8
SMM      51
Name: A4CUID, dtype: int64
Total patients (MGUS, SMM, MM, PCL) = 493


In [6]:
# average age for pts
round(MM_PCL_SMM_MGUS.groupby(['Status'])['profile.age'].describe(),2)

,count,mean,std,min,25%,50%,75%,max
Status,,,,,,,,
alive,435.0,65.21,9.65,37.0,60.00,66.0,72.00,91.0
deceased,58.0,68.67,10.01,39.0,63.25,71.0,75.75,83.0


In [7]:
#average age for pts - alive
round(MM_PCL_SMM_MGUS.groupby(['Status'])['profile.age'].describe(),2)

,count,mean,std,min,25%,50%,75%,max
Status,,,,,,,,
alive,435.0,65.21,9.65,37.0,60.00,66.0,72.00,91.0
deceased,58.0,68.67,10.01,39.0,63.25,71.0,75.75,83.0


In [8]:
# gender % for pts
print(MM_PCL_SMM_MGUS.groupby(['profile.gender'])['A4CUID'].count())


profile.gender
female    242
male      251
Name: A4CUID, dtype: int64


In [9]:
# states represented, (-1 because there is a category called 'Outside of US')
statecount = MM_PCL_SMM_MGUS['location_state'].nunique()
print("number of states represented minus one = {}".format(statecount))

# counting countries
countries = MM_PCL_SMM_MGUS.loc[MM_PCL_SMM_MGUS['location_state'].str.contains('Out of US')]
countries = countries['location_city'].nunique()
print("number of non-US countries represented = {}".format(countries))

number of states represented minus one = 43
number of non-US countries represented = 8


In [10]:
# patients who had a pre existing condition

MMptsonly = MM_PCL_SMM_MGUS.loc[(MM_PCL_SMM_MGUS.diagnosis == 'MM')].reset_index(drop=True)
MMptsonly.loc[(pd.notnull(MMptsonly['diagnosis_MGUS']))].count()

A4CUID            42
diagnosis         42
diagnosis_MGUS    42
diagnosis_SMM     12
diagnosis_MM      42
profile.age       42
profile.gender    42
dashboardlink     42
Status            42
primary_clinic    42
#_of_clinics      42
last_update       42
location_city     42
location_state    42
QCCA              42
dtype: int64

### LOT Analysis

For the LOT analysis we will exclude Smoldering patients and only analyze MM and PCL patients.

In [11]:
# Identify MM and PCL patients in ptkey

MM_PCL = ptkey.loc[~ptkey['diagnosis'].str.contains('SMM|Myelodysplastic|Waldenstrom|MGUS|Amyloidosis|Lymphoma')]

In [12]:
# Merge the identified patients with the LOT 

LOT_ptkey = pd.merge(LOTdata, MM_PCL, on=['A4CUID'])

In [13]:
# Count of A4CUID by order of therapy
LOT_ptkey.groupby(['Order'])['A4CUID'].count()

Order
0       4
1     414
2     315
3     215
4     143
5      94
6      76
7      59
8      35
9      24
10     16
11     11
12      9
13      6
14      5
15      3
16      1
Name: A4CUID, dtype: int64

In [14]:
# Separate out order 1 therapy for further analysis

firstline = LOT_ptkey.loc[LOT_ptkey['Order'] == 1]
unique_LOT = firstline['Line of Therapy'].nunique()
print("{} patients received unique first line therapy".format(unique_LOT))

89 patients received unique first line therapy


In [15]:
# Count number of patients per therapy
firstline_group = pd.DataFrame(firstline.groupby(['Line of Therapy'])['A4CUID'].count().sort_values(ascending=False))
firstline_group

,A4CUID
Line of Therapy,
RVD,92
CyVD,50
RVD + Auto-SCT + R,41
RD,29
VD,22
...,...
RD + Allo-SCT,1
RD + Auto-SCT,1
RD + Auto-SCT + R,1


In [16]:
# Get info on pts who received a unique first line therapy

uniquefirstline = firstline_group.loc[firstline_group['A4CUID'] == 1]
uniquedf = pd.merge(uniquefirstline, firstline, on=['Line of Therapy'])
unique1stlinetherapypts = uniquedf['Line of Therapy'].count()
print("{} patients received a first line therapy that was not received by others".format(unique1stlinetherapypts))

62 patients received a first line therapy that was not received by others


In [ ]:
uniquedf.to_csv('uniquefirstline.csv',index=False)

In [17]:
# number of patients who were a part of clinical trial
numberofclinicaltrials = uniquedf['NCT number'].count()
print("{} patients in the unique first line therapy category participated in a clinical trial".format(numberofclinicaltrials))

17 patients in the unique first line therapy category participated in a clinical trial


In [18]:
#Response breakdown in unique 1st line therapy

uniquedf.groupby(['Response'])['A4CUID_x'].count()

Response
CR      10
MR       2
PR       6
SD       3
VGPR    19
sCR     14
Name: A4CUID_x, dtype: int64

In [19]:
# number of patients who received 6th and beyond, categorized by responses
numberofptsbeyond6th = LOT_ptkey.loc[LOT_ptkey['Order'] >= 6]['A4CUID'].count()
print("{} patients received 6th line and above".format(numberofptsbeyond6th))

LOT_ptkey.loc[LOT_ptkey['Order'] >= 6].groupby(['Response'])['A4CUID'].count()

245 patients received 6th line and above


Response
CR       2
MR      21
PD      49
PR      41
SD      52
VGPR    31
sCR      7
Name: A4CUID, dtype: int64

In [20]:
# successive line of therapy analysis table

responsesbyorder = pd.pivot_table(LOT_ptkey, values='A4CUID', index='Order',columns=['Response'], 
                                  aggfunc=len, margins=True, dropna=False)
responsesbyorder = pd.DataFrame(responsesbyorder)
responsesbyorder = responsesbyorder.fillna(0)

# calculations

responsesbyorder['% Achieving sCR'] = round(((responsesbyorder['sCR'] / responsesbyorder['All']) * 100), 1)
responsesbyorder['ORR (%)'] = round(((responsesbyorder['PR'] + responsesbyorder['VGPR'] +
                                      responsesbyorder['CR'] + responsesbyorder['sCR'])
                                     / responsesbyorder['All'])*100,1)

responsesbyorder['CBR (%)'] = round(((responsesbyorder['SD'] + responsesbyorder['MR'] +
                                      responsesbyorder['PR'] + responsesbyorder['VGPR'] + responsesbyorder['CR'] +
                                      responsesbyorder['sCR']) / responsesbyorder['All'])*100, 1)


# output pivot table
responsesbyorder_table = responsesbyorder[['All', '% Achieving sCR', 'ORR (%)', 'CBR (%)']]
responsesbyorder_table = responsesbyorder_table.rename(columns={'All':'Number of Patients'})
responsesbyorder_table = responsesbyorder_table.iloc[0:16]


In [21]:
# calculating time interval

LOT_ptkey['Treatment Start'] = pd.to_datetime(LOT_ptkey['Treatment Start'])
LOT_ptkey['End Treatment'] = pd.to_datetime(LOT_ptkey['End Treatment'])

timeinterval = LOT_ptkey[['A4CUID', 'Order', 'Line of Therapy', 'Response', 'NCT number', 'Treatment Start', 'End Treatment']]

timeinterval = timeinterval.rename(columns={'Treatment Start':'txstart', 'End Treatment':'txend'})

timeinterval['txgap'] = timeinterval.groupby('A4CUID')['txstart'].diff()

In [22]:
# function to determine gap mean and gap std for each time interval 

orders = [2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13, 14, 15, 16]

def assigngapmean(x):
    txgap = timeinterval.loc[timeinterval['Order'] == x]['txgap'].mean()
    return(txgap)

    
def assigngapstd(x):
    txgapstd = timeinterval.loc[timeinterval['Order'] == x]['txgap'].std()
    return(txgapstd)

def assigngapmed(x):
    txgapmed = timeinterval.loc[timeinterval['Order'] == x]['txgap'].median()
    return(txgapmed)


In [23]:
txgapmean={}
txgapstd={}
txgapmed={}

for x in orders:
    txgapmean["{0}".format(x)] = assigngapmean(x)
    txgapstd["{0}".format(x)] = assigngapstd(x)
    txgapmed["{0}".format(x)] = assigngapmed(x)


In [24]:
# merge mean, med, std data together

txgap_mean = pd.DataFrame(txgapmean.items(), columns=['Order', 'txgapmean'])
txgap_std = pd.DataFrame(txgapstd.items(), columns=['Order', 'txgapstd'])
txgap_med = pd.DataFrame(txgapmed.items(), columns=['Order', 'txgapmed'])

txgap_df = pd.merge(txgap_mean, txgap_std, on=['Order'], how='left')
txgap_df = pd.merge(txgap_df, txgap_med, on=['Order'], how='left')

In [25]:
def organizedays(x):
    txgap_df[x] = txgap_df[x].astype(str)
    txgap_df[x]= txgap_df[x].str.split(" ", n = 1, expand = True) 
    return txgap_df[x]

names = ['txgapmean', 'txgapstd', 'txgapmed']

for name in names:
    txgap_df[name] = organizedays(name)

In [26]:
# merge gap data and responsesbyorder table

responsesbyorder_table = responsesbyorder_table.reset_index(level=['Order'])
responsesbyorder_table['Order'] = responsesbyorder_table['Order'].astype(str)
completedf = pd.merge(responsesbyorder_table, txgap_df, how='left', on=['Order'])

for name in names:
    completedf[name] = completedf[name].str.replace('NaT', 'N/A')

completedf['txgapmean'] = completedf['txgapmean'].shift(-1)
completedf['txgapstd'] = completedf['txgapstd'].shift(-1)
completedf['txgapmed'] = completedf['txgapmed'].shift(-1)

completedf = completedf[['Order', 'Number of Patients', '% Achieving sCR', 'ORR (%)', 'CBR (%)', 'txgapmean',
                        'txgapmed', 'txgapstd']]

completedf = completedf.rename(columns = {'txgapmean':'Days until next line of therapy (mean)',
                                          'txgapstd':'Days until next line of therapy (std)',
                                         'txgapmed':'Days until next line of therapy (median)'})

completedf = completedf.replace(np.NaN, 'N/A')

In [27]:
#deceased pts

deceasedpts = LOT_ptkey.loc[LOT_ptkey['Status'] == 'deceased']
deceasedptslastline = deceasedpts.groupby(['A4CUID']).last()
deceasedptslastline['deceased'] = 'Y'

deceasedptslastline.sort_values(['Order'])

deceasedcount = deceasedptslastline.groupby(['Order'])['deceased'].count()
deceasedcount = pd.DataFrame(deceasedcount)
deceasedcount.reset_index(level=0, inplace=True)

In [28]:
completedf['Order'] = completedf['Order'].astype(int)

completedf = pd.merge(completedf, deceasedcount, on=['Order'])

completedf = completedf[['Order', 'Number of Patients', '% Achieving sCR', 'ORR (%)', 'CBR (%)',
                         'deceased', 'Days until next line of therapy (mean)', 
                         'Days until next line of therapy (median)',
                         'Days until next line of therapy (std)']]


completedf = completedf.rename(columns={'deceased':'Interval Deaths',
                                       'Order':'Line of Therapy'})